1.3 Pytorch Neural Networks(神经网络)

### 1. 使用torch.nn包来构建神经网络

> 由上一讲了解了autograd, nn包依赖 autograd 包来定义模型并求导。一个nn.Module包含各个层和一个forward(input)方法，该方法返回output。

![Alexnet网络示例](./img/alexnet20220517154203.png "网络模型")

> 这是一个简单的前馈神经网络， 它接受一个输入， 然后一层接着一层地传递， 最后输出计算结果。

> 神经网络的典型训练过程如下：

    1. 定义包含一些可学习参数（或者叫权重）的神经网络模型；
    2. 在数据集上迭代；
    3. 通过神经网络处理输入；
    4. 计算损失（输出结果和正确值的差值大小）；
    5. 将梯度反向传播回网格的参数；
    6. 更新网格的参数， 主要使用如下简单的更新原则： weight = weight - learning_rate*gradient

### 2. 定义网络

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten all dimensions except the batch dimension
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        

In [4]:
net = Net()

In [5]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

> 您只需要定义forward函数， 就可以使用autograd为您自动定义的backward函数（计算梯度）。您可以在forward函数中使用任何张量操作。

> 模型的可以学习参数由 net.parameters() 返回

In [6]:
net.parameters()

<generator object Module.parameters at 0x000002C7C064AE40>

In [23]:
# list(net.parameters())

In [9]:
len(list(net.parameters()))

10

In [10]:
params = list(net.parameters())

In [11]:
params[0].size()

torch.Size([6, 1, 5, 5])

In [12]:
params[1].size()

torch.Size([6])

In [13]:
params[2].size()

torch.Size([16, 6, 5, 5])

In [14]:
params[3].size()

torch.Size([16])

In [15]:
params[4].size()

torch.Size([120, 400])

In [16]:
params[5].size()

torch.Size([120])

In [18]:
params[6].size()

torch.Size([84, 120])

In [19]:
params[7].size()

torch.Size([84])

In [20]:
params[8].size()

torch.Size([10, 84])

In [21]:
params[9].size()

torch.Size([10])

> 让我们尝试一个 32 * 32 的随机输入。 注意该网络的预期输入大小（LeNet）为32*32 。要在MNIST数据集上使用此网络，请将图像从数据集中调整为32*32。

In [24]:
input = torch.randn(1, 1, 32, 32)

In [25]:
input

tensor([[[[-0.6879,  1.3057,  1.3715,  ..., -1.4337, -2.0822,  0.2898],
          [ 0.1196, -0.6910,  0.4534,  ..., -0.5788, -0.1214,  0.3163],
          [-0.3782,  1.4453,  0.3271,  ..., -1.2923, -0.4578,  1.2522],
          ...,
          [-1.3988, -0.5945,  1.2240,  ..., -1.2059, -1.1987, -0.2059],
          [-0.3658,  0.0668,  1.0944,  ...,  0.4230,  0.7303, -0.4681],
          [ 1.5490, -0.5790, -0.2589,  ...,  1.5986,  0.0743, -0.1137]]]])

In [26]:
out = net(input)

D:\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [27]:
out

tensor([[-0.0751, -0.0740, -0.0316,  0.0862, -0.0508, -0.0083, -0.0213, -0.0867,
          0.0594, -0.0045]], grad_fn=<AddmmBackward>)

> 使用随机梯度将所有参数和反向传播的缓冲区归零：

In [28]:
net.zero_grad()

In [29]:
out.backward(torch.randn(1, 10))

> torch.nn 仅支持小批量。整个torch.nn 包仅支持作为微型样本而不是单个样本的输入。

> 例如： nn.Conv2d 将采用 nSamples * nChannels * Height * Width 的4D张量。

> 如果您只有一个样本， 只需要使用 input.unsqueeze(0) 添加一个假批量尺寸。

> 在继续之前，我们回顾一下到目前为止看到的所有类。

* torch.Tensor 一个多维数组， 支持如 backward()的自动微分操作。同样，保持相对于张量的梯度。
* nn.Module 神经网络模块， 封装参数的便捷方法， 并带有将其移动到GPU、导出、加载等的帮助器。
* nn.Parameter 一种张量，即将其分配为 Module 的属性时， 自动注册为参数。
* autograd.Fuction 实现自动微分操作的正向和反向定义。每个Tensor操作都会创建至少一个Function节点， 该节点连接到创建Tensor的函数，并且编码器历史记录
    

### 3. 损失函数

> 损失函数采用一对（输出， 目标）输入， 并计算一个值， 该值估计输出与目标之间的距离。

> nn 包含多种不同的损失函数。一个简单的损失函数是：nn.MSELoss, 它计算输入和目标之间的均方误差。

> *例如：*

In [31]:
output = net(input)
# a dummy target, for example
target = torch.randn(10)
# make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)


D:\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [32]:
loss

tensor(1.5263, grad_fn=<MseLossBackward>)

In [33]:
# 现在， 如果使用 .grad_fn 属性向后跟随 loss， 您将看到一个计算图， 如下所示：

In [35]:
loss.grad_fn

In [36]:
loss.grad_fn.next_functions

((<AddmmBackward at 0x2c7c1366460>, 0), (None, 0))

In [38]:
loss.grad_fn.next_functions[0][0]

In [39]:
loss.grad_fn.next_functions[0][0].next_functions

((<AccumulateGrad at 0x2c7c0db5490>, 0),
 (<ReluBackward0 at 0x2c7c0fab8e0>, 0),
 (<TBackward at 0x2c7c0fab400>, 0))

In [40]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

### 4. 反向传播

In [41]:
# zeroes the gradient buffers os all parameters
net.zero_grad()

In [42]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [44]:
loss.backward()

In [45]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0010,  0.0077, -0.0088,  0.0165, -0.0166, -0.0211])


### 5. 更新权重

> weight = weight - learning_rate * gradient

In [46]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [47]:
import torch.optim as optim

In [48]:
# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [49]:
# in your training loop:
# zero the gradient buffers
optimizer.zero_grad()

In [50]:
output = net(input)

In [51]:
loss = criterion(output, target)

D:\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [52]:
loss.backward()
# does the update
optimizer.step()

> 注意

> 观察如何使用optimizer.zero_grad()将梯度缓冲区手动设置为零。 这是因为如反向传播部分中所述累积了梯度。